In [3]:
import requests
import pandas as pd
import re
import pickle
import os
import warnings
from tqdm import tnrange, tqdm_notebook
from PIL import Image

In [19]:
url_tpl = "https://www.wikiart.org/en/paintings-by-style/{}?json=2&layout=new&page={}&resultType=masonry"
artist = "ukiyo-e"

# Find image URLs

In [20]:
page = 1
all_paintings_df = pd.DataFrame()

while True:
    url = url_tpl.format(artist, page)
    resp = requests.get(url = url)
    paintings_list = resp.json()["Paintings"]
    if paintings_list == None:
        break
    paintings_df = pd.DataFrame(paintings_list)
    all_paintings_df = pd.concat([all_paintings_df, paintings_df])
    page = page + 1
    
all_paintings_df = all_paintings_df.reset_index(drop=True)

# Download images

In [25]:
num = 0

for im_url in all_paintings_df.image:
    name = re.findall("[^/]+$", im_url)[0]
    img = requests.get(im_url).content
    with open("../images/" + artist + "_raw/" + "%04d" % num + "-" + name, 'wb') as f:
        f.write(img)
    num = num + 1

# Convert images

## HD conversion

In [4]:
input_dir = "../images/ukiyo-e_raw"
output_dir = "../images/ukiyo-e_fullhd"

target_width = 1920
target_height = 1080

image_files = os.listdir(input_dir)
image_files = list(filter(lambda x: x.endswith('.jpg'), image_files))

Image.MAX_IMAGE_PIXELS = 999999999999 # cause there are some large files

for im in tqdm_notebook(image_files):
    input_loc = input_dir + '/' + im
    output_loc = output_dir + '/' + im

    img = Image.open(input_loc)
    img = img.convert('RGB') # as it seems some images are not JPG

    if (img.width < target_width or img.height < target_height) and (img.height < target_width or img.width < target_height):
        continue
    
    if img.width < img.height:
        img = img.rotate(90, expand=1)

    scale = max(target_width / img.width, target_height / img.height)
    new_width = round(scale * img.width)
    new_height = round(scale * img.height)

    img_resized = img.resize((new_width, new_height), Image.LANCZOS)

    crop_left = int((img_resized.width - target_width)/2)
    crop_top = int((img_resized.height - target_height)/2)

    img_cropped = img_resized.crop((crop_left, crop_top, crop_left + target_width, crop_top + target_height))

    img_cropped.save(output_loc, quality = 100)